In [53]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value inout data : Real; //flow properties => inout
            ref inout commIF : CommunicationInterface; //uggghhhh not sure how this one works (seems recursive ?!)
        }
    }
    
    
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        part groundStation : 'Ground Station' {
            port redefines groundCommunicationInterface;
        }
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port redefines spaceCommunicationInterface;
        }
        
        interface def CommunicationLink { //defining CommunicationLink type 
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
        }
        
        interface : CommunicationLink connect //creating the link
            groundPort => groundStation::groundCommunicationInterface to //why is it blue though ?
            spacePort  => spacecraftVehicle::spaceCommunicationInterface;
    }
    
    
    
    
    block 'Ground Station'{
        port inout groundCommunicationInterface : CommunicationInterface;
    }
    
    
    
    block 'Spacecraft Vehicle'{
        port inout spaceCommunicationInterface : CommunicationInterface;
       
        value data : Real = 102400; //is it the correct notation for default values?
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
    
    
    
}




Package Structure (6cef008a)


In [31]:
%show Structure::Interfaces

Package Interfaces (782d93bd)
  [Membership] PortDefinition CommunicationInterface (3491668a)
    [FeatureMembership] ValueProperty data (c918b94)
      [FeatureTyping] DataType Real (77bc2e16)
      [Subsetting] ValueProperty values (a451491)
    [Superclassing] PortDefinition Port (4c6007fb)
